## Impacto del plan 401(k) en Riqueza Financiera


En la clase pasada, como aplicación del Método Double ML, consideramos la estimación del efecto de la elegibilidad  en planes 401(k) sobre los activos acumulados. Los planes 401(k) son cuentas de pensión patrocinadas por los empleadores. El problema clave al determinar el efecto de la participación en planes 401(k) sobre los activos acumulados es la **heterogeneidad de los ahorradores**, junto con el hecho de que la decisión de inscribirse en un 401(k) **no es aleatoria**.

El reto de esta semana consiste en determinar el **efecto causal de la participación en un plan 401k (p401) sobre la riqueza financiera (net_tfa)**

### Datos


In [ ]:
!pip install wget
!pip install formulaic

In [ ]:
# Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

from sklearn.model_selection import cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV, LinearRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.base import TransformerMixin, BaseEstimator, clone
from IPython.display import Markdown
import wget
import seaborn as sns
from formulaic import Formula
import warnings
warnings.simplefilter('ignore')
np.random.seed(1234)

In [ ]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/401k.csv"
data = pd.read_csv(file)

In [ ]:
data.head()

In [ ]:
readme = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/401k.md"
filename = wget.download(readme)
Markdown(open(filename, 'r').read())

## Preguntas 1 y 2

1. ¿Está la participación en el programa está asociada con riqueza financiera? Justifica tu respuesta

2. ¿Cuál es el "Average predictive effect" (APE) de *p401* en *net_tfa*?

Notemos que estos valores estimados están sesgados ya que no toman en cuenta la heterogeneidad de los ahorradores

In [ ]:
#Seleccionando variables
y = data['net_tfa'].values
D = data['e401'].values
D2 = data['p401'].values
D3 = data['a401'].values
X = data.drop(['e401', 'p401', 'a401', 'tw', 'tfa', 'net_tfa', 'tfa_he',
               'hval', 'hmort', 'hequity',
               'nifa', 'net_nifa', 'net_n401', 'ira',
               'dum91', 'icat', 'ecat', 'zhat',
               'i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7',
               'a1', 'a2', 'a3', 'a4', 'a5'], axis=1)
X.columns

### Definimos una transformación para el feature engineering de los controles

In [ ]:
class FormulaTransformer(TransformerMixin, BaseEstimator):

    def __init__(self, formula, array=False):
        self.formula = formula
        self.array = array

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        df = Formula(self.formula).get_model_matrix(X)
        if self.array:
            return df.values
        return df

In [ ]:
transformer = FormulaTransformer("0 + poly(age, degree=6, raw=True) + poly(inc, degree=8, raw=True) "
                                 "+ poly(educ, degree=4, raw=True) + poly(fsize, degree=2, raw=True) "
                                 "+ male + marr + twoearn + db + pira + hown", array=True)

## Calculando el ATE de la participación en el plan 401k en los Activos Financieros Netos

Queremos estimadores válidos del efecto promedio del tratamiento (ATE) de  `p401` y  en `net_tfa`. Utilizamos modelos de ML para estimar las funciones $g_0$ y $m_0$ en el siguiente modelo

\begin{eqnarray}
 &  Y = D\theta_0 + g_0(X) + \zeta,  &  E[\zeta \mid D,X]= 0,\\
 & D = m_0(X) +  V,   &  E[V \mid X] = 0.
\end{eqnarray}

## Preguntas 3, 4 y 5

3. Calcula el efecto promedio causal (ATE) de la participación en el plan 401k sobre la riqueza financiera.

4. ¿Cómo compara este efecto con el ATE de la elegibilidad a participar en este plan sobre la riqueza financiera (clase pasada)?

5. Justifica tu elección de los modelos de Machine Learning empleados para la estimación de *`y`* y de *`D`*